In [39]:
import meraki
import pandas as pd
import time
from datetime import datetime
import sqlite3

from meraki_utils import connect_to_meraki

In [40]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [41]:
dashboard = connect_to_meraki()

In [42]:
records = []

In [43]:
# Get organizations
orgs = dashboard.organizations.getOrganizations()
org_id = orgs[0]['id']

In [44]:
# Get networks
networks = dashboard.organizations.getOrganizationNetworks(organizationId=org_id,
                                                           productTypes=['switch'])

In [45]:
# Create network lookup table
network_lookup = {net['id']: net['name'] for net in networks}

In [46]:
# Get switches
switches = dashboard.organizations.getOrganizationDevices(organizationId=org_id,
                                                         productTypes=['switch'])

In [57]:
for switch in switches:
    print(f"Processing {switch['name']}")
    serial = switch['serial']
    network_id = switch.get('networkId')
    network_name = network_lookup.get(network_id, "Unbounded") if network_id else "No Network"
    device_name = switch.get('name', serial)

    # Get each ports status
    try:
        ports = dashboard.switch.getDeviceSwitchPortsStatuses(serial=serial,
                                                              timespan=86400)
    except meraki.APIError as e:
        print(f"  ⚠️ Error on {serial}: {e}")
        continue

    if not ports:
        print(f"  ⚠️ No ports returned for {serial} (empty response)")
        continue

    for port in ports:
        print(f"  Processing port {port['portId']}")
        traffic = port.get('usageInKb', {}).get('trafficInKbps', port.get('usageInKb', {}))
        recv_mbps = round(traffic.get('recv', 0) / 1000, 3)
        sent_mbps = round(traffic.get('sent', 0) / 1000, 3)
        total_mbps = round(traffic.get('total', 0) / 1000, 3)

        # CDP/LLDP neighbour
        neighbour = ""
        if port.get('cdp') or port.get('lldp'):
            info = port.get('cdp') or port.get('lldp')
            neigh_dev = info.get('deviceId') or info.get('systemName', '')
            neigh_port = info.get('portId') or ''
            neighbour = f"{neigh_dev}:{neigh_port}" if neigh_dev else ""

        # Add data to records
        records.append({
            'timestamp': pd.Timestamp.utcnow(),
            'network': network_name,
            'device': device_name,
            'serial': serial,
            'port': port['portId'],#
            'enabled': port['enabled'],
            'status': port['status'],
            'speed': port.get('speed', ''),
            'duplex': port.get('duplex', ''),
            'recv_mbps': recv_mbps,
            'sent_mbps': sent_mbps,
            'total_mbps': total_mbps,
            'errors': len(port.get('errors', [])),
            'discards': len(port.get('discards', [])),
            'neighbour': neighbour.strip()
        })

    time.sleep(0.15) # Pause for rate limit

Processing J-SW-HUT01
  Processing port 1
  Processing port 2
  Processing port 3
  Processing port 4
  Processing port 5
  Processing port 6
  Processing port 7
  Processing port 8
  Processing port 9
  Processing port 10
Processing C-SW-PAV-01
  Processing port 1
  Processing port 2
  Processing port 3
  Processing port 4
  Processing port 5
  Processing port 6
  Processing port 7
  Processing port 8
  Processing port 9
  Processing port 10


KeyboardInterrupt: 

In [51]:
# Create dataframe
df = pd.DataFrame(records)

df

In [49]:
print(f"Collected {len(df)} port samples from {df['serial'].nunique()} switches.")
df.to_csv('../csv_exports/port_utilization_draft.csv', index=False, header=True, sep=',')

Collected 2926 port samples from 73 switches.


In [50]:
# Top 20 busiest ports
print("\nTop 20 busiest ports:")
df.nlargest(20, 'total_mbps')[['device', 'port', 'total_mbps', 'neighbour']]


Top 20 busiest ports:


,device,port,total_mbps,neighbour
0,J-SW-HUT01,1,0.0,
1,J-SW-HUT01,2,0.0,
2,J-SW-HUT01,3,0.0,
3,J-SW-HUT01,4,0.0,
4,J-SW-HUT01,5,0.0,
5,J-SW-HUT01,6,0.0,
6,J-SW-HUT01,7,0.0,
7,J-SW-HUT01,8,0.0,
8,J-SW-HUT01,9,0.0,
9,J-SW-HUT01,10,0.0,
